# Knee Force Analysis

Trying to figure out why squats feel so much harder than standing. Turns out the forces on your knee change dramatically with angle.

**Assumptions & Limitations:**
- Static analysis only (no acceleration)
- Ignores ligament contributions and muscle co-contraction
- Single-plane 2D model (real movement is 3D)
- Moment arm values are estimates from literature

**Abigail Wu**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from src.biomechanics import *

print('✓ Loaded')

## Scenario 1: Just Standing

Baseline case - what's the knee dealing with when you're not doing anything?

In [ ]:
body_mass = 70  # kg
shank = create_segment('shank', body_mass, length=0.43)

knee_pos = np.array([0, 0.5])
ankle_pos = np.array([0, 0.05])

# Ground pushes up with half body weight per leg (symmetric standing)
ground_force = np.array([0, body_mass * 9.81 / 2])

knee_force = calculate_joint_force(shank, knee_pos, ankle_pos, ground_force)

print(f'Body mass: {body_mass} kg')
print(f'Shank mass: {shank.mass:.2f} kg (about 4.7% of body mass)')
print(f'Ground force per leg: {ground_force[1]:.1f} N')
print(f'\nKnee compressive force: {abs(knee_force[1]):.1f} N')
print(f'= {abs(knee_force[1])/body_weight_force(body_mass):.2f}x body weight')
print('\nMakes sense - slightly more than GRF because of shank weight')

## Scenario 2: Squatting

What changes when you squat to 90°? Forces get way higher.

In [ ]:
# Squat position - knee bent to ~90°
knee_pos_squat = np.array([0, 0.3])
ankle_pos_squat = np.array([0.15, 0.05])  # Foot slides forward

# GRF increases during squat (literature values: 1.2-1.6x BW)
# Using conservative estimate
ground_force_squat = np.array([0, body_mass * 9.81 * 0.6])

knee_force_squat = calculate_joint_force(shank, knee_pos_squat, ankle_pos_squat, ground_force_squat)

print(f'Squatting:')
print(f'Knee force: {abs(knee_force_squat[1]):.1f} N')
print(f'= {abs(knee_force_squat[1])/body_weight_force(body_mass):.2f}x body weight')
print(f'\nIncrease from standing: {(abs(knee_force_squat[1])/abs(knee_force[1]) - 1)*100:.1f}%')
print('\nNote: This is probably underestimated since we\'re ignoring')
print('muscle co-contraction and the actual forces during descent/ascent')

## Quadriceps Muscle Force

How hard do the quads need to pull to generate knee extension torque?

In [ ]:
# Knee extension moment during squat
# Literature reports 40-60 N·m for bodyweight squats
knee_moment_needed = 50  # N·m (middle of range)

# Quadriceps moment arm varies with knee angle
# ~4-6cm is typical range, using 5cm here
quad_moment_arm = 0.05  # meters

quad_force = calculate_muscle_force(knee_moment_needed, quad_moment_arm)

print(f'Required knee extension moment: {knee_moment_needed} N⋅m')
print(f'Quadriceps moment arm: {quad_moment_arm*100:.1f} cm')
print(f'\nQuadriceps force needed: {quad_force:.0f} N')
print(f'= {quad_force/body_weight_force(body_mass):.1f}x body weight')
print('\nThis is surprisingly high! The small moment arm means')
print('muscles need huge forces to generate modest torques.')

## Force vs Knee Angle

Exploring how knee angle affects force requirements.

In [ ]:
angles = []
forces = []

# Simulate different squat depths
for knee_height in np.linspace(0.5, 0.2, 10):
    knee = np.array([0, knee_height])
    hip = np.array([0, knee_height + 0.4])
    ankle = np.array([0.1, 0.05])
    
    angle = calculate_angle(hip, knee, ankle)
    angles.append(angle)
    
    # Rough force estimate (keeping GRF constant for comparison)
    grf = np.array([0, 500])
    kf = calculate_joint_force(shank, knee, ankle, grf)
    forces.append(abs(kf[1]))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(angles, linewidth=2, marker='o')
ax1.set_xlabel('Squat Depth', fontsize=11)
ax1.set_ylabel('Knee Angle (degrees)', fontsize=11)
ax1.set_title('Knee Angle During Squat', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axhline(90, color='r', linestyle='--', alpha=0.5, label='90° (parallel)')
ax1.legend()

ax2.scatter(angles, forces, s=60, alpha=0.7)
ax2.set_xlabel('Knee Angle (degrees)', fontsize=11)
ax2.set_ylabel('Knee Force (N)', fontsize=11)
ax2.set_title('Force vs Angle', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'Force range: {min(forces):.0f} - {max(forces):.0f} N')
print('\nInteresting: forces don\'t change linearly with angle.')
print('The geometry matters a lot.')

## What I Learned

1. **Forces are huge** - Even standing still puts significant load on joints. Squatting multiplies this.

2. **Moment arms are critical** - Small moment arms mean muscles need enormous forces to generate torque. This is why leverage matters so much in biomechanics.

3. **These are underestimates** - Real forces are probably 20-30% higher because:
   - Muscles co-contract for stability
   - Ligaments provide additional forces
   - Dynamic acceleration adds extra load
   - 3D geometry is more complex than 2D models

4. **Posture matters** - Small changes in joint position can dramatically change force distribution.

## Next Steps

- Add muscle co-contraction estimates
- Include acceleration terms for dynamic movements
- Compare these simplified calculations to published EMG and force plate data
- Extend to 3D analysis